In [1]:
import numpy as np
import pandas as pd

path = r"C:\Users\HP\OneDrive\Escritorio\David Guzzi\DiTella\MEC\Materias\2025\2025 1T\[MT10] Series de Tiempo\Clases prácticas\Práctica 2-20250504\termduffie.txt"
df = pd.read_csv(path, delimiter="\t", decimal=".")
df.head()

,Name,curv,dr,r10y,r1y,r2y,r3m,r5y,r6m,spread1,spread10,spread2,spread5
0,NaN,-0.000750,NaN,0.006241,0.004396,0.004721,0.003952,0.005416,0.004169,0.000444,0.002290,0.000770,0.001464
1,NaN,-0.000370,0.000600,0.006006,0.004837,0.005093,0.004551,0.005546,0.004672,0.000286,0.001454,0.000542,0.000995
2,NaN,-0.000427,-0.000062,0.006690,0.005066,0.005376,0.004490,0.006013,0.004808,0.000577,0.002200,0.000887,0.001523
3,NaN,-0.000318,0.000609,0.006337,0.005364,0.005558,0.005098,0.005895,0.005226,0.000266,0.001239,0.000460,0.000797
4,NaN,-0.000769,-0.000054,0.006827,0.005288,0.005551,0.005044,0.006153,0.005138,0.000244,0.001783,0.000507,0.001109


In [2]:
df = df.drop(columns=['Name'])
df.head(1)

,curv,dr,r10y,r1y,r2y,r3m,r5y,r6m,spread1,spread10,spread2,spread5
0,-0.00075,NaN,0.006241,0.004396,0.004721,0.003952,0.005416,0.004169,0.000444,0.00229,0.00077,0.001464


In [9]:
from statsmodels.tsa.vector_ar.vecm import select_coint_rank

# Asegurarse de que no haya NaNs
data = df[['r10y', 'r1y', 'r2y', 'r3m', 'r5y', 'r6m']].copy()

rank_test = select_coint_rank(data, det_order=0, k_ar_diff=8, method='trace', signif=0.05)
print(rank_test.summary())

Johansen cointegration test using trace test statistic with 5% significance level
r_0 r_1 test statistic critical value
-------------------------------------
  0   6          143.2          95.75
  1   6          94.57          69.82
  2   6          60.06          47.85
  3   6          33.78          29.80
  4   6          17.33          15.49
  5   6          2.678          3.841
-------------------------------------


In [10]:
rank_test.rank

5

In [11]:
from statsmodels.tsa.vector_ar.vecm import VECM

vecm = VECM(data, k_ar_diff=7, coint_rank=rank_test.rank, deterministic='ci')
vecm_res = vecm.fit()

# Cointegrating relationships (beta)
print("Cointegration vectors (beta):")
print(vecm_res.beta)

# Alpha (ajuste del ECM)
print("\nAdjustment coefficients (alpha):")
print(vecm_res.alpha)

# Resumen detallado con betas, alphas, y ecuaciones de corto plazo
print("\nSummary:")
print(vecm_res.summary())

Cointegration vectors (beta):
[[ 1.00000000e+00  1.85094566e-17 -1.36652532e-16 -2.30733738e-17
  -5.21426378e-16]
 [ 8.07880068e-16  1.00000000e+00  3.98003253e-16 -7.45872697e-17
   1.18989447e-15]
 [-5.46839908e-16  6.37144576e-17  1.00000000e+00  1.21442431e-16
  -8.53085121e-16]
 [ 9.55700551e-18 -6.95160705e-17  2.02530743e-16  1.00000000e+00
   6.19789411e-16]
 [-1.97693979e-16 -7.11465403e-17  9.45251679e-17 -4.18126630e-18
   1.00000000e+00]
 [-1.03446736e+00 -1.01048208e+00 -1.01849350e+00 -9.90114918e-01
  -1.02198034e+00]]

Adjustment coefficients (alpha):
[[ 1.37339236 -2.19314754  1.56531866 -0.92851853 -2.45140877]
 [ 2.7128852  -3.61090913  3.68367734 -1.59602684 -4.81138717]
 [ 2.39646393 -2.93358295  2.62254018 -1.43526406 -4.06160734]
 [ 3.17551705 -1.90782518  3.7722589  -1.92673103 -5.71835416]
 [ 2.0359686  -3.03293971  2.56316509 -1.27641608 -3.63783914]
 [ 2.9049185  -2.23755688  3.49720188 -1.4834276  -5.17420955]]

Summary:
Det. terms outside the coint. relati

In [15]:
# Además del VAR estructural, se suma ECV completo junto con el uso de restricciones. Final clase 5 y principio clase 6.